In [46]:
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import statsmodels.tsa.api as tsa
from statsmodels.graphics import tsaplots

import torch
import torch.nn as nn


In [4]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
data = yf.download("AAPL")

if ticker.info['regularMarketPrice'] is None:
    print("Error")
else:
    print(ticker.info['regularMarketPrice'])


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


268.47


In [6]:
dividends = ticker.dividends

info = ticker.info

print(dividends)
print(info.get("dividendRate"), info.get("dividendYield"))

Date
1987-05-11 00:00:00-04:00    0.000536
1987-08-10 00:00:00-04:00    0.000536
1987-11-17 00:00:00-05:00    0.000714
1988-02-12 00:00:00-05:00    0.000714
1988-05-16 00:00:00-04:00    0.000714
                               ...   
2024-08-12 00:00:00-04:00    0.250000
2024-11-08 00:00:00-05:00    0.250000
2025-02-10 00:00:00-05:00    0.250000
2025-05-12 00:00:00-04:00    0.260000
2025-08-11 00:00:00-04:00    0.260000
Name: Dividends, Length: 88, dtype: float64
1.04 0.39


In [7]:
import yfinance as yf

t = yf.Ticker('AAPL')
t.ticker

'AAPL'

In [47]:
from Stock import Stock
import pandas as pd

stock = Stock('GOOG')
# path = os.getcwd() + f'/data/dividends_{stock.ticker.ticker}.csv'
dividends_df = pd.DataFrame(stock.get_dividends())
dividends_df

,Dividends
Date,
2024-06-10 00:00:00-04:00,0.20
2024-09-09 00:00:00-04:00,0.20
2024-12-09 00:00:00-05:00,0.20
2025-03-10 00:00:00-04:00,0.20
2025-06-09 00:00:00-04:00,0.21
2025-09-08 00:00:00-04:00,0.21


In [49]:
from Database.database import engine
from Database.models import Holding
from sqlalchemy import select

with engine.connect() as conn:
    holdings_df = pd.read_sql(select(Holding), conn)

dividends_df = pd.DataFrame(columns=['Date','Dividends'])
dividends_df = stock.get_dividends().reset_index()

for ticker in holdings_df['ticker']:
    stock = Stock(ticker)

    divs_df = stock.get_dividends().reset_index()
    divs_df['Company'] = ticker

    pd.concat([dividends_df, divs_df])

dividends_df

2025-11-11 13:18:13,171 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-11 13:18:13,173 INFO sqlalchemy.engine.Engine SELECT holdings.id, holdings.ticker, holdings."holdingType", holdings."holdingSize", holdings."dateAdded", holdings."dateEdited" 
FROM holdings
2025-11-11 13:18:13,174 INFO sqlalchemy.engine.Engine [cached since 8317s ago] ()
2025-11-11 13:18:13,175 INFO sqlalchemy.engine.Engine ROLLBACK


,Date,Dividends
0,1987-05-11 00:00:00-04:00,0.000536
1,1987-08-10 00:00:00-04:00,0.000536
2,1987-11-17 00:00:00-05:00,0.000714
3,1988-02-12 00:00:00-05:00,0.000714
4,1988-05-16 00:00:00-04:00,0.000714
...,...,...
84,2024-11-08 00:00:00-05:00,0.250000
85,2025-02-10 00:00:00-05:00,0.250000
86,2025-05-12 00:00:00-04:00,0.260000
87,2025-08-11 00:00:00-04:00,0.260000


In [10]:
holdings_df = pd.DataFrame([], columns=['Ticker', 'Holding Type', 'Holding Size'])

holdings_df.to_csv('holdings_df.csv')


In [12]:
def message_handler(message):
    print("Received message: ", message)


In [13]:
def market_info(market_name):
    market = yf.Market(market_name)
    status = market.status
    summary = market.summary

    print(status)
    print(summary)

In [14]:
def options(ticker_name):
    ticker = yf.Ticker(ticker_name)

    return ticker.option_chain(ticker.options[0]).calls


In [15]:
def general_info(ticker_name):
    ticker_name = "AAPL"

    ticker = yf.Ticker(ticker_name)

    print(ticker.info)



In [ ]:
ticker = yf.Ticker("AAPL")
tickers = yf.Tickers(' aapl msft goog')

In [ ]:
analyst_price_targets = ticker.analyst_price_targets
calendar = ticker.calendar


In [ ]:
appl_df = yf.download(['AAPL'])
appl_df.to_csv('AAPL_df.csv')
# print(appl_df)

plt.plot(appl_df['Close'])
plt.show()

tsaplots.acf(appl_df['Close']) # also try appl_df

# tsaplots.acf(appl_df)

plt.show()